In [ ]:
import os
import pandas as pd
import requests
import datetime

##########################################################
from dotenv import load_dotenv

import os

load_dotenv()
ASU_key = os.environ.get("ASU_key")
file_name = os.environ.get("file_name")
LLM_url = os.environ.get("LLM_url")

now = datetime.datetime.now()
formatted_datetime = now.strftime("%Y_%m_%d_%H")

In [36]:
directory_path = 'C:\\programming_projects\\RAG_fine_tune\\RAG_pipeline_ASU_website\\data\\'
print(directory_path + 'chunked_' +file_name)
chunked_df = pd.read_csv(directory_path + 'chunked_' +file_name, nrows=100)

C:\programming_projects\RAG_fine_tune\RAG_pipeline_ASU_website\data\chunked_cleaned_ASU_webpage_04_07_2025.csv


In [37]:
chunked_df.columns

Index(['url', 'depth', 'title', 'topic', 'orig_word_count', 'orig_char_count',
       'page_text', 'filename', 'cleaned_text', 'langauage', 'chunked_text',
       'chunked_word_count', 'chunked_char_count'],
      dtype='object')

In [38]:
chunked_df[['cleaned_text', 'chunked_text']].head(10)

,cleaned_text,chunked_text
0,Nondegree requirementsTo be eligible to take c...,Nondegree requirementsTo be eligible to take c...
1,ASU Online empowers first year students to tak...,ASU Online empowers first year students to tak...
2,Becky received her BSc in Ocean and Earth Scie...,Becky received her BSc in Ocean and Earth Scie...
3,Brett is a Postdoctoral Scientist in the Coral...,Brett is a Postdoctoral Scientist in the Coral...
4,Rachel is a microbial oceanographer who invest...,Rachel is a microbial oceanographer who invest...
5,Rachel is a microbial oceanographer who invest...,F.E. B. Felts M. Breitbart P. Salamon R.A. Edw...
6,Jessica completed her Bachelor of Science in m...,Jessica completed her Bachelor of Science in m...
7,Rebecca joined BIOS in September 2021 as a sea...,Rebecca joined BIOS in September 2021 as a sea...
8,Dr. Carlson earned his BA degree at Colby Coll...,Dr. Carlson earned his BA degree at Colby Coll...
9,Steven Giovannoni has a PhD from the Universit...,Steven Giovannoni has a PhD from the Universit...


In [ ]:
df_out = pd.DataFrame([])
for i in range(0, 2):
    print(i)

    cleaned_string = chunked_df['cleaned_text'].loc[i]

    chunk = chunked_df['chunked_text'].loc[i]
    title = chunked_df['title'].loc[i]
    url = chunked_df['url'].loc[i]
    chunked_word_count = chunked_df['chunked_word_count'].loc[i]
    orig_word_count = chunked_df['orig_word_count'].loc[i]
    ##########################################
    ## text type
    bearer_token = ASU_key
    json_payload = {
        "query": "what is the topic of the following text from: {cleaned_string}? only respond with the topic, no other text.".format(cleaned_string=cleaned_string),
        "model_provider": "gcp-deepmind",
        "model_name": "geminiflash2",
    }
    headers = {
        "Authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    try:
        response = requests.post(LLM_url, headers=headers, json=json_payload)
        response.raise_for_status()
        result_document_section = response.json().get("response")
        # print("result:", result_document_section)
    except requests.exceptions.RequestException as e:
        print(f"API request error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

    ##########################################
    ## questions

    query = """given that the following text from the webpage {title} on url {url}, here is a text chunk limited to 500 words:\n {chunk}\n\n 
                what are some good questions to ask about the text chunk? Please respond with a question and 3 different answers for each question.  There should be a total of 3 questions, with having 3 different answers (for a total of 9 unique answers).

                the questions need to be well defined. Try to use the text as much as possible when crafting the answer. Answers need to be at least 2 sentences long. Do not use the phrase "The text," and avoid similar language. Rephrase the question in the answer.

                Please use the following format for the response:

                **Question 1:**
                **Question 1 Answer 1:**
                **Question 1 Answer 2:**
                **Question 1 Answer 3:**

                **Question 2:**
                **Question 2 Answer 1:**
                **Question 2 Answer 2:**
                **Question 2 Answer 3:**

                **Question 3:**
                **Question 3 Answer 1:**
                **Question 3 Answer 2:**
                **Question 3 Answer 3:**
                """.format(
                    title = title,
                    url = url,
                    chunk=chunk)

    json_payload = {
        "query": query,
        "model_provider": "gcp-deepmind",
        "model_name": "geminiflash2",
    }
    headers = {
        "Authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    try:
        response = requests.post(LLM_url, headers=headers, json=json_payload)
        response.raise_for_status()
        result = response.json().get("response")
        # print("result:", result)
    except requests.exceptions.RequestException as e:
        print(f"API request error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")
    
    ###################################################
    ## save out
    parts = result.split("**")  # Split the string at **
    parts_no = [[2, 4], [2, 6], [2,8], [10, 12], [10, 14], [10, 16], [18, 20], [18, 22], [18, 24]]


    for i in range(0, 9):
        try:
            pt1 = parts_no[i][0]
            pt2 = parts_no[i][1]

            Q1 = pd.DataFrame(data={'section':[result_document_section],
                                    'title':[title],
                                    'ur': [url],
                                    'document_type':['web page'],
                                    'chunked_word_count':[chunked_word_count],
                                    'orig_word_count':[orig_word_count],
                                    'contex': [chunk],
                                    'page_whole': [cleaned_string],
                                    'question':[parts[pt1]],
                                    'answer':[parts[pt2]]
                                    })
            df_out = pd.concat([Q1, df_out], ignore_index=True)
            df_out.to_csv(directory_path+f'silver_data_{formatted_datetime}.csv', index=False)

        except Exception as e:
            print(f"Unexpected error: {e}")
            # print("result:", result)


0
1


In [41]:

df_out.head()

,section,title,document_type,chunked_word_count,orig_word_count,contex,page_whole,question,answer
0,ASU Online bachelor's degrees,First Year Students ASU Online,web page,55,55,ASU Online empowers first year students to tak...,ASU Online empowers first year students to tak...,\nWhat is the purpose of ASU Online's support ...,\nASU Online's support services aim to help st...
1,ASU Online bachelor's degrees,First Year Students ASU Online,web page,55,55,ASU Online empowers first year students to tak...,ASU Online empowers first year students to tak...,\nWhat is the purpose of ASU Online's support ...,\nThe purpose of ASU Online's support services...
2,ASU Online bachelor's degrees,First Year Students ASU Online,web page,55,55,ASU Online empowers first year students to tak...,ASU Online empowers first year students to tak...,\nWhat is the purpose of ASU Online's support ...,\nASU Online's support services are designed t...
3,ASU Online bachelor's degrees,First Year Students ASU Online,web page,55,55,ASU Online empowers first year students to tak...,ASU Online empowers first year students to tak...,\nWhat does Arizona State University offer onl...,\nArizona State University ensures that online...
4,ASU Online bachelor's degrees,First Year Students ASU Online,web page,55,55,ASU Online empowers first year students to tak...,ASU Online empowers first year students to tak...,\nWhat does Arizona State University offer onl...,\nArizona State University offers the option t...
